<a href="https://colab.research.google.com/github/thyarles/unb-fmc-nlp/blob/main/aula_1/notes_lets_build_the_gpt_tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Let's build the GPT Tokenizer
Notes from the video https://www.youtube.com/watch?v=zduSFxRajkE.

* Most of the problem we see on the LLM are from Tokenizers (like do simple math wrong).
* The unicode has three types, UTF-8, UTF-16, and UTF-32. The UTF-8 is the standard because it the only one that has variable length. For latin characters, the UTF-16 add zero word on every letter, and UTF-32 add two zero words.
* We can't use the Unicode to tokenizer because it has a huge code space (about 150 thousand).  

In [ ]:
# To check the UTF-8 value
[ord(x) for x in "Charles."]

[67, 104, 97, 114, 108, 101, 115, 46]

In [ ]:
# To check the UTFs 8, 16 and 32.
print("%s\n%s\n%s" %
(
  list("Charles.".encode("utf-8")),
  list("Charles.".encode("utf-16")),
  list("Charles.".encode("utf-32")))
)

[67, 104, 97, 114, 108, 101, 115, 46]
[255, 254, 67, 0, 104, 0, 97, 0, 114, 0, 108, 0, 101, 0, 115, 0, 46, 0]
[255, 254, 0, 0, 67, 0, 0, 0, 104, 0, 0, 0, 97, 0, 0, 0, 114, 0, 0, 0, 108, 0, 0, 0, 101, 0, 0, 0, 115, 0, 0, 0, 46, 0, 0, 0]


# Byte pair encoding

In [11]:
# Example: aaabdaaabac (vocabulary size = eleven, four tokens)
#          -> find the pair that occurs more frequently and replace it with a single token
#          Z = aa, Y = ab, X = zy -> XdXac (vocabulary size = seven, five tokens)

# Fancy chars consume more bytes, that's why the code points is less than tokens
text = "Հայերեն Shqip العربية Български Català 中文简体 Hrvatski Česky Dansk Nederlands English."
tokens = text.encode("utf-8")     # raw bytes
tokens = list(map(int, tokens))   # integers
print("The text has %d code points and %d tokens." % (len(text), len(tokens)))

The text has 84 code points and 117 tokens.


In [14]:
# Let's find the most frequent value

# Using the pythonic way
def get_stats_pythonic(ids):
  counts = {}
  for pair in zip(ids, ids[1:]):
    counts[pair] = counts.get(pair, 0) + 1
  return counts

# Using human way
def get_stats_human(ids):
    counts = {}
    for i in range(len(ids) - 1):
        pair = (ids[i], ids[i + 1])
        if pair in counts:
            counts[pair] += 1
        else:
            counts[pair] = 1
    return counts

stats = get_stats_pythonic(tokens)
print(sorted(((v, k) for k,v in stats.items()), reverse=True))

[(3, (115, 107)), (2, (213, 165)), (2, (128, 213)), (2, (97, 116)), (2, (97, 110)), (1, (231, 174)), (1, (230, 150)), (1, (228, 189)), (1, (228, 184)), (1, (217, 138)), (1, (217, 132)), (1, (216, 185)), (1, (216, 177)), (1, (216, 169)), (1, (216, 168)), (1, (216, 167)), (1, (214, 128)), (1, (213, 182)), (1, (213, 181)), (1, (213, 161)), (1, (213, 128)), (1, (209, 138)), (1, (209, 129)), (1, (209, 128)), (1, (208, 187)), (1, (208, 186)), (1, (208, 184)), (1, (208, 179)), (1, (208, 176)), (1, (208, 145)), (1, (196, 140)), (1, (195, 160)), (1, (189, 147)), (1, (187, 208)), (1, (186, 208)), (1, (185, 216)), (1, (184, 173)), (1, (184, 32)), (1, (182, 32)), (1, (181, 213)), (1, (179, 208)), (1, (177, 216)), (1, (176, 209)), (1, (174, 128)), (1, (173, 230)), (1, (169, 32)), (1, (168, 217)), (1, (167, 217)), (1, (165, 214)), (1, (165, 213)), (1, (161, 213)), (1, (160, 32)), (1, (150, 135)), (1, (147, 32)), (1, (145, 209)), (1, (140, 101)), (1, (138, 216)), (1, (138, 208)), (1, (135, 231)), (1,